In [15]:
from data_parser import VelParser
from entropy_compressor import ZCompressor
from quantizator import Quantizator
from wavelet_compressor import WaveletCompressor

import os
import shutil
import numpy as np
from typing import Dict, List

COMPRESS_PATH = 'compressed_low'
DECOMPRESS_PATH = 'decompressed'
IMAGE_NAME = 'low.png'
MESH_KEYS = ['faces', 'neighbour', 'owner', 'points']
PTURHO_KEYS = ['p', 'T', 'U', 'rho']


class Compressor:
    def __init__(self):
        if not os.path.exists(COMPRESS_PATH):
            os.mkdir(COMPRESS_PATH)
        self.entropy_compressor = ZCompressor()
        self.quantizator = Quantizator()
        self.wavelet_compressor = WaveletCompressor()
        self.parser = VelParser()

    def total_compress(self, data: np.ndarray) -> List[np.ndarray | np.float64]:
        wavelet_compressed = self.wavelet_compressor.wavelet_transform(data)[0]
        quantized, q_step, min_val = self.quantizator.uniform_quantization(wavelet_compressed)
        entropy_compressed = self.entropy_compressor.compress(quantized).hex()
        return [entropy_compressed, q_step, min_val]

    def save_mesh(self, mesh_data: Dict[str, np.ndarray]) -> None:
        # faces, neighbour, owner, pts = mesh_data.values()
        with open(COMPRESS_PATH + '/' + 'mesh', mode='w+') as f:
            print(list(mesh_data.keys())[:-1])
            for (key, value) in list(mesh_data.items())[:-1]:
                compressed_ndarray, q_step, min_val = self.total_compress(value)
                f.write(f'{q_step} {min_val}\n')
                f.write(str(compressed_ndarray) + '\n')

    def compress_and_save(self, model: str, base_path: str, dim: str):
        vels = list(map(str, sorted(self.parser.get_vels(
            model=model,
            base_path=base_path,
            dimpath=dim,
        ), key=lambda x: str(x))))
        mesh_data = self.parser.get_mesh_data(
            path_to_vel=vels[0],
        )
        self.save_mesh(mesh_data=mesh_data)
        
        for vel in vels:
            data = self.parser.get_vel_data(
                path_to_vel=vel,
            )
            print(data)
            vellast = vel.split('/')[-1]
            with open(COMPRESS_PATH + '/' + vellast, mode='w+') as f:
                times = list(data.keys())
                f.write(f'{";".join(times)}\n')
                for (_, value) in list(data.items())[1:]:
                    p, t, u, rho = value
                    print(p, t, u, rho)
                    # print(value)
                    compressed_p, p_q_step, p_min_val = self.total_compress(p)
                    compressed_t, t_q_step, t_min_val = self.total_compress(t)
                    compressed_u, u_q_step, u_min_val = self.total_compress(u)
                    compressed_rho, rho_q_step, rho_min_val = self.total_compress(rho)
                    f.write(f'{p_q_step} {p_min_val}\n')
                    f.write(f'{str(compressed_p)}' + '\n')

                    f.write(f'{t_q_step} {t_min_val}\n')
                    f.write(f'{str(compressed_t)}' + '\n')

                    f.write(f'{u_q_step} {u_min_val}\n')
                    f.write(f'{str(compressed_u)}' + '\n')

                    f.write(f'{rho_q_step} {rho_min_val}\n')
                    f.write(f'{str(compressed_rho)}' + '\n')
    
    def copy_not_compressable(self, model: str, base_path: str, dim: str):
        vels = list(map(str, sorted(self.parser.get_vels(
            model=model,
            base_path=base_path,
            dimpath=dim,
        ), key=lambda x: str(x))))
        zero_vel = vels[0]
        thermophysical_path = f'{zero_vel}/constant/thermophysicalProperties'
        turbulence_props_path = f'{zero_vel}/constant/turbulenceProperties'
        boundary_path = f'{zero_vel}/constant/polyMesh/boundary'
        image_path = f'{zero_vel}/{IMAGE_NAME}'
        system_path = f'{zero_vel}/system'
        destination_path = f'{COMPRESS_PATH}/not_compressable'
        # os.mkdir(path=destination_path)
        shutil.copytree(system_path, destination_path)
        shutil.copy(thermophysical_path, destination_path)
        shutil.copy(boundary_path, destination_path)
        if os.path.exists(image_path):
            shutil.copy(image_path, destination_path)
        shutil.copy(turbulence_props_path, destination_path)

In [18]:
compressor = Compressor()
data = compressor.compress_and_save(
    model='data_wage',
    base_path='data',
    dim='low_dim',
)

['faces', 'neighbour', 'owner', 'points']


In [5]:
compressor = Compressor()
compressor.copy_not_compressable(
    model='data_wage',
    base_path='data',
    dim='low_dim',
)

In [ ]:
vels = list(map(str, sorted(compressor.parser.get_vels(
    model='data_wage',
    base_path='data',
    dimpath='low_dim',
), key=lambda x: str(x))))
mesh_data = compressor.parser.get_mesh_data(
    path_to_vel=vels[0],
)

In [17]:
compressor.total_compress(mesh_data['faces'])

NameError: name 'mesh_data' is not defined

In [29]:
compressor.total_compress(mesh_data['neighbour'])

['111000010111111001111101011101110101101110011011110011111111110100010011110100110110010001111001011001011110100110111011111101111011101011110110010011011011111010001101010100001110100110001011100011100100111110011101011001001110111111100011101101100011110011000001100001110011111010101010110110111111010000000000000000000000000000000000000',
 0.002994060516357422,
 -30.03125]

In [30]:
compressor.total_compress(mesh_data['owner'])

['11110010110001111000111101100000111111101100011101001110011111101010000000010011100001110101011000101111111001110110101111001100010101110100011011110111101001000100010001011000001101000111001111110101011011100000101111111101110101011001100000100101100111111001011100100101010001101001100011010011001110111001001110111011011100010001010000100001110110000011110100010011100010011100101110000011101100100101101010001110110011011011011110000110001111111011001010011010010000110001011111101010111111110011100010111100111001100101001011101011',
 0.002654552459716797,
 -13.15625]

In [31]:
compressor.total_compress(mesh_data['points'])

['00110011000011000110000110001111000100011111010010110010000100000110000001111100001011100000111001001010100100100011110011011101010111010001101100000000110111001111101101001010011000110011110010110101011111100001100101110111111000101101111111100010001100000010010001111000100110011101001111001101011011101011101010110100011111111111111110010011111110100100110010001001101010011000110011111001100011011111000111011000011011100011110001110011111111010011101011011011110000111011111101011001011111011100110010101001101011001011100001101010001111011111100010010001010010110011000000001101111011110111101101100100011110010110011100011011001101100100100111011011001101011111001000010100001111001011010111010100111001000001011010011110110011111110101001101000110011100100110111110010000001010100011010001010111101000010110010001000000010101001110010111000100010000111011100010111110011011111101110111111101110001010100011000011101011001111010111110100011101100110011110110110011010101010110101001100111100